<a href="https://colab.research.google.com/github/ii200400/Tensorflow_Tutorial/blob/master/04%20-%20Neural%20Network%20Basic/02_Deep_NN(2_1ver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

포유류인지 조류인지 분류하는 간단한 신경망 모델을 구현

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

TensorFlow 2.x selected.


## 데이터 정의

In [ ]:
# [털, 날개]
x_data = np.array(
    [[0., 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]], dtype=np.float32)

# [기타, 포유류, 조류]
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

## 신경망 모델 구성

### 가중치,편향 정의

In [ ]:
# 첫번째 가중치의 차원은 [특성, 히든 레이어의 뉴런갯수] -> [2, 10] 으로 정한다.
W1 = tf.Variable(tf.random.uniform([2, 10], -1., 1.))
# 두번째 가중치의 차원을 [첫번째 히든 레이어의 뉴런 갯수, 분류 갯수] -> [10, 3] 으로 정한다.
W2 = tf.Variable(tf.random.uniform([10, 3], -1., 1.))

# 편향을 각각 각 레이어의 출력 갯수로 설정한다.
# b1 은 히든 레이어의 뉴런 갯수로, b2 는 최종 결과값 즉, 분류 갯수인 3으로 설정한다.
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

### 모델 함수와 활성화 함수

In [ ]:
def model(x):
  # 신경망의 히든 레이어에 가중치 W1과 편향 b1을 적용한다.
  L1 = tf.add(tf.matmul(x, W1), b1)
  L1 = tf.nn.relu(L1)
  
  # 히든레이어에 두번째 가중치 W2와 편향 b2를 적용하여 3개의 출력값을 계산한다.
  output = tf.add(tf.matmul(L1, W2), b2)
  return output

### 비용함수

tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(predicted_y), axis=1)) 대신\
tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=model(x))) 사용

그런데 이름이 너무 길어서 더 복잡해 보인닼ㅋㅋ

In [ ]:
# 텐서플로우에서 기본적으로 제공되는 크로스 엔트로피 함수를 이용해
# 복잡한 수식을 사용하지 않고도 최적화를 위한 비용 함수를 다음처럼 간단하게 적용할 수 있다.
def get_cost(x, y):
  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=model(x)))

### 최적화 함수

In [ ]:
# 최적화 함수로 아담(Adam)을 사용한다.
optimizer = tf.optimizers.Adam(learning_rate=0.01)

### 훈련 함수

In [ ]:
def train(x, y):
  with tf.GradientTape() as t:
    cost = get_cost(x, y)
  
  grads = t.gradient(cost, [W1, W2, b1, b2])
  optimizer.apply_gradients(zip(grads, [W1, W2, b1, b2]))

  return cost

## 모델 학습 및 테스트

### 모델 학습

In [ ]:
for step in range(100):
    cost = train(x_data, y_data)

    if (step + 1) % 10 == 0:
        print(step + 1, cost)

10 tf.Tensor(0.044922814, shape=(), dtype=float32)
20 tf.Tensor(0.036141146, shape=(), dtype=float32)
30 tf.Tensor(0.029748304, shape=(), dtype=float32)
40 tf.Tensor(0.024962857, shape=(), dtype=float32)
50 tf.Tensor(0.021287823, shape=(), dtype=float32)
60 tf.Tensor(0.01839974, shape=(), dtype=float32)
70 tf.Tensor(0.016085573, shape=(), dtype=float32)
80 tf.Tensor(0.014198626, shape=(), dtype=float32)
90 tf.Tensor(0.012632993, shape=(), dtype=float32)
100 tf.Tensor(0.011318095, shape=(), dtype=float32)


### 모델 테스트

In [ ]:
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류

prediction = tf.argmax(model(x_data), 1)
target = tf.argmax(y_data, 1)
print('예측값:', prediction)
print('실제값:', target)

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % (accuracy * 100))

예측값: tf.Tensor([0 1 2 0 0 2], shape=(6,), dtype=int64)
실제값: tf.Tensor([0 1 2 0 0 2], shape=(6,), dtype=int64)
정확도: 100.00


# 전체 코드

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array(
    [[0., 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]], dtype=np.float32)

# [기타, 포유류, 조류]
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

#########
# 신경망 모델 구성
######

W1 = tf.Variable(tf.random.uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random.uniform([10, 3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

optimizer = tf.optimizers.Adam(learning_rate=0.01)

def model(x):
  L1 = tf.add(tf.matmul(x, W1), b1)
  L1 = tf.nn.relu(L1)

  return tf.add(tf.matmul(L1, W2), b2)

def get_cost(x, y):
  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=model(x)))

def train(x, y):
  with tf.GradientTape() as t:
    cost = get_cost(x, y)
  
  grads = t.gradient(cost, [W1, W2, b1, b2])
  optimizer.apply_gradients(zip(grads, [W1, W2, b1, b2]))


#########
# 신경망 모델 학습
######

for step in range(100):
    cost = train(x_data, y_data)

    if (step + 1) % 10 == 0:
        print(step + 1, cost)


#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######

prediction = tf.argmax(model(x_data), 1)
target = tf.argmax(y_data, 1)
print('예측값:', prediction)
print('실제값:', target)

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % (accuracy * 100))

10 None
20 None
30 None
40 None
50 None
60 None
70 None
80 None
90 None
100 None
예측값: tf.Tensor([0 1 2 0 0 2], shape=(6,), dtype=int64)
실제값: tf.Tensor([0 1 2 0 0 2], shape=(6,), dtype=int64)
정확도: 100.00
